<a href="https://colab.research.google.com/github/kub522/gsdgf/blob/main/COMP_351_Genomic%20Sequence%20Analysis_Final%20Code_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install biopython

In [ ]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from Bio import SeqIO
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Chrom15 = '/content/drive/MyDrive/COMP351/Chrom15.fasta'    # Seq for Chromosome 15
Alb = '/content/drive/MyDrive/COMP351/Alb.fasta'      # Seq for Albinism

df = SeqIO.parse(Chrom15, "fasta") # Using Biopython, we can parse through the fasta file which contains the sequence for chromosome 15
dg = SeqIO.parse(Alb, "fasta") # Using Biopython, we can parse through the fasta file which contains the sequence for Albinism


In [ ]:
for record in dg: #loop to print the ALb fasta out in 50 character long lines
  Albseq_str = record.seq
  for i in range(0, len(Albseq_str), 50):
    print(Albseq_str[i:i+50])

AATCCTCCTCTTCTCCCTTCCTGAAGCTGCACGCTGCAGTAAGAGCACAG
CAGAAATGCAGACAAAAGGGGGCCAAACATGGGCGAGAAGGGCTCTGTTG
CTCGGCATCCTGTGGGCCACTGCACATCTGCCTCTCTCAGGGACCTCCCT
GCCCCAACGTCTCCCAAGGGCCACAGGAAATAGCACCCAATGTGTTATTT
CTCCATCATCGGAGTTTCCCGAAGGGTTTTTCACGAGACAGGAGCGCAGA
GATGGAGGCATCATAATCTATTTCCTAATTATCGTTTACATGTTCATGGC
CATATCTATTGTCTGTGATGAATACTTCCTACCCTCCCTGGAAATCATCA
GTGAATCCCTTGGATTGTCTCAGGATGTTGCAGGCACAACTTTCATGGCA
GCGGGCAGTTCAGCTCCTGAATTAGTTACTGCTTTCCTAGGTGTATTTAT
CACAAAGGGAGATATTGGCATTAGCACCATCCTTGGATCTGCAATTTATA
ATCTCCTTGGCATCTGTGCTGCCTGTGGTTTGCTATCTAATACGGTCTCA
ACACTATCATGTTGGCCCCTATTCAGAGACTGTGCAGCGTACACAATTAG
TGCAGCAGCAGTTCTTGGTATAATATATGACAACCAAGTTTACTGGTATG
AAGGGGCTTTACTGCTTTTGATATATGGATTGTATGTTTTGGTGCTGTGT
TTTGACATTAAAATTAACCAATATATTATAAAGAAATGCAGTCCTTGCTG
CGCCTGTCTTGCCAAAGCTATGGAGAGAAGTGAACAACAGCCACTGATGG
GCTGGGAAGATGAAGGTCAACCATTCATTCGTCGGCAATCAAGAACTGAT
AGTGGAATATTTTATGAAGATTCTGGCTACTCTCAGCTCTCTATAAGTTT
ACATGGCCTTAGTCAGGTTTCTGAAGATCCACCAAGTGTTTTCAACATGC
CTGAAGCAGACTTAAAAAGAATTTTTTGGGT

In [ ]:
char_to_int = {char: i for i, char in enumerate("ACGT")} # int encoding to make nucleotides correspond to intergers A = 0, C = 1, G = 2, and T = 3
Albintseq = [char_to_int[char] for char in record.seq]  # create a list of our now coded nucleotides
print(Albintseq)

[0, 0, 3, 1, 1, 3, 1, 1, 3, 1, 3, 3, 1, 3, 1, 1, 1, 3, 3, 1, 1, 3, 2, 0, 0, 2, 1, 3, 2, 1, 0, 1, 2, 1, 3, 2, 1, 0, 2, 3, 0, 0, 2, 0, 2, 1, 0, 1, 0, 2, 1, 0, 2, 0, 0, 0, 3, 2, 1, 0, 2, 0, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 0, 3, 2, 2, 2, 1, 2, 0, 2, 0, 0, 2, 2, 2, 1, 3, 1, 3, 2, 3, 3, 2, 1, 3, 1, 2, 2, 1, 0, 3, 1, 1, 3, 2, 3, 2, 2, 2, 1, 1, 0, 1, 3, 2, 1, 0, 1, 0, 3, 1, 3, 2, 1, 1, 3, 1, 3, 1, 3, 1, 0, 2, 2, 2, 0, 1, 1, 3, 1, 1, 1, 3, 2, 1, 1, 1, 1, 0, 0, 1, 2, 3, 1, 3, 1, 1, 1, 0, 0, 2, 2, 2, 1, 1, 0, 1, 0, 2, 2, 0, 0, 0, 3, 0, 2, 1, 0, 1, 1, 1, 0, 0, 3, 2, 3, 2, 3, 3, 0, 3, 3, 3, 1, 3, 1, 1, 0, 3, 1, 0, 3, 1, 2, 2, 0, 2, 3, 3, 3, 1, 1, 1, 2, 0, 0, 2, 2, 2, 3, 3, 3, 3, 3, 1, 0, 1, 2, 0, 2, 0, 1, 0, 2, 2, 0, 2, 1, 2, 1, 0, 2, 0, 2, 0, 3, 2, 2, 0, 2, 2, 1, 0, 3, 1, 0, 3, 0, 0, 3, 1, 3, 0, 3, 3, 3, 1, 1, 3, 0, 0, 3, 3, 0, 3, 1, 2, 3, 3, 3, 0, 1, 0, 3, 2, 3, 3, 1, 0, 3, 2, 2, 1, 1, 0, 3, 0, 3, 1, 3, 0, 3, 3, 2, 3, 1, 3, 2, 3, 2, 0, 3, 2, 0, 0, 3, 0, 1, 3, 3, 1, 1, 3, 0, 1, 1, 

In [ ]:
for record in df: #loop to print the Chrom15 fasta out in 50 character long lines
  Chrom15seq_str = record.seq     #WARNING MUCH LONGER
  for i in range(0, len(Chrom15seq_str), 50):
    print(Chrom15seq_str[i:i+50])

TTCCCAACAGGATCTCCCACCAGCCCAGCTTTTCTGTATAGGCTCCGGCC
TCTGCTCATCCAAGGTAAGCGGTAGCCTCTGAGGTTGGAGGATGGAGAAG
GTGGAATCAGGCCAATTGAAAAGGACAGCCAGGCTGCCATTTCTCTAGAA
TCAAAATGGAGGTTGTGGAGTACGGGATCCCTGAGTTCTCTCCGAGCTTC
TTGGGTAGCCTGAATAAATACTTGCCCCCAAGGATGGACCTCAGCTGTCC
CTCAGATGGAGCTGAGAGGTAGAGCCATATGTCTGTGGAGGTCTATTGGT
TCTATCAGGGGAGCAGAAGGCAGAGGGTGAGAGGCAAGCACTTGGGTGAG
GAGTTGATTGGCTAAGCCAACAACATGCATGCTTCGCTGAGGATGTCCCA
GTGAGGGGACTCAGCAGCAGAGGGAGGAGCAAGGGGAATGGTGAGAACCT
TTACCATAAGAGCCAGTATGTGGCTCAGACGCACTGTAGGACTGAGGTTG
GGGCATGAGAAAGGATATCTAAGAGGTTTACAGCACACTAAGTCACACCA
ATAAGCTGACCCTGCTCTGCCTCTGCCTCCATATGGATGGGTCTTCGGGC
ATTTAGAGACTCCAGGCTCACTATAGCACAAGGGTCTCCAGTTGGAGAAA
CAGATGCTGTAACTGGGGCCACAGTATAAAGAGAGCCCAGACATTGAACT
GTCAACACAGAAGCCTGGCACACTGGAACTGGCAGTCCAGCTGGGAACAA
GGGGTAGAGGCTGAGGCCACTAAGTCAACTGAGGCAGGAGACATAGGAGC
TAAAGCAGCTGAAGGGTGCAGGACAGCTGGGGGGTCTGAAGTGGGCCTCA
TGCCCAGAGCTATGAAGTCAGGGGCTGTAGCCTAGGAGCCTTGGAAGCCA
GCTGGCAAGCTGTGGCCCAAAGACGCTGACTCACCAGGAGGGGGCAGCTG
GAGCCAGGCACTCCTAAGGTTTCCAGGAAGG

In [ ]:
#Enumeration and further processing of Chrom 15

char_to_int = {char: i for i, char in enumerate("ACGT")} # int encoding to make nucleotides correspond to intergers A = 0, C = 1, G = 2, and T = 3
Chrom15intseq = [char_to_int[char] for char in record.seq]  # create a list of our now coded nucleotides
print(Chrom15intseq)

[3, 3, 1, 1, 1, 0, 0, 1, 0, 2, 2, 0, 3, 1, 3, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 0, 2, 1, 3, 3, 3, 3, 1, 3, 2, 3, 0, 3, 0, 2, 2, 1, 3, 1, 1, 2, 2, 1, 1, 3, 1, 3, 2, 1, 3, 1, 0, 3, 1, 1, 0, 0, 2, 2, 3, 0, 0, 2, 1, 2, 2, 3, 0, 2, 1, 1, 3, 1, 3, 2, 0, 2, 2, 3, 3, 2, 2, 0, 2, 2, 0, 3, 2, 2, 0, 2, 0, 0, 2, 2, 3, 2, 2, 0, 0, 3, 1, 0, 2, 2, 1, 1, 0, 0, 3, 3, 2, 0, 0, 0, 0, 2, 2, 0, 1, 0, 2, 1, 1, 0, 2, 2, 1, 3, 2, 1, 1, 0, 3, 3, 3, 1, 3, 1, 3, 0, 2, 0, 0, 3, 1, 0, 0, 0, 0, 3, 2, 2, 0, 2, 2, 3, 3, 2, 3, 2, 2, 0, 2, 3, 0, 1, 2, 2, 2, 0, 3, 1, 1, 1, 3, 2, 0, 2, 3, 3, 1, 3, 1, 3, 1, 1, 2, 0, 2, 1, 3, 3, 1, 3, 3, 2, 2, 2, 3, 0, 2, 1, 1, 3, 2, 0, 0, 3, 0, 0, 0, 3, 0, 1, 3, 3, 2, 1, 1, 1, 1, 1, 0, 0, 2, 2, 0, 3, 2, 2, 0, 1, 1, 3, 1, 0, 2, 1, 3, 2, 3, 1, 1, 1, 3, 1, 0, 2, 0, 3, 2, 2, 0, 2, 1, 3, 2, 0, 2, 0, 2, 2, 3, 0, 2, 0, 2, 1, 1, 0, 3, 0, 3, 2, 3, 1, 3, 2, 3, 2, 2, 0, 2, 2, 3, 1, 3, 0, 3, 3, 2, 2, 3, 3, 1, 3, 0, 3, 1, 0, 2, 2, 2, 2, 0, 2, 1, 0, 2, 0, 0, 2, 2, 1, 0, 2, 0, 2, 2, 2, 3, 2, 0, 2, 0, 2, 

In [ ]:
char_sequence_series = pd.Series(Chrom15intseq, name="base")

X = pd.get_dummies(char_sequence_series, drop_first= True)
y_rawdata = (char_sequence_series == char_to_int["C"]).astype(int)
y = y_rawdata

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42) # training on 80% of data and leave the remaining 20% for testing

In [ ]:

# 4. Build a simple neural network model

model_dropout = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])
model_dropout.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_dropout.summary()


In [ ]:
# 5. Train the model
# Convert y_train and y_test to NumPy arrays as that we need for the validation split
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

history = model.fit(X_train, y_train_np, epochs= 10, batch_size= 64,
                    validation_split=0.2, verbose=1)

y_prob = model.predict(X_test)
y_pred = (y_prob > 0.5).astype("int32")
print("Test accuracy:", accuracy_score(y_test_np, y_pred))

In [ ]:
history.history.keys()



In [ ]:

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:


# Plot training & validation accuracy values
plt.plot(history.history['loss'], label='Train error')
plt.plot(history.history['val_loss'], label='Validation error')
plt.title('Model error Over Epochs')
plt.ylabel('Error')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# 6. Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# 5. Train the model
# Convert y_train and y_test to NumPy arrays bc Tensor needs that apparently
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

history = model.fit(X_train, y_train_np, epochs= 10, batch_size= 64,
                    validation_split=0.2, verbose=1)


# 6. Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))